In [1]:
### Implementation of 'from lifelines.utils import concordance_index'
# because of no internet access(!pip install lifelines) when inferencing with kaggle
# src: https://github.com/CamDavidsonPilon/lifelines/blob/47afb1c1a272b0f03e0c8ca00e63df27eb2a0560/lifelines/utils/concordance.py

import numpy as np


class _BTree:

    """A simple balanced binary order statistic tree to help compute the concordance.

    When computing the concordance, we know all the values the tree will ever contain. That
    condition simplifies this tree a lot. It means that instead of crazy AVL/red-black shenanigans
    we can simply do the following:

    - Store the final tree in flattened form in an array (so node i's children are 2i+1, 2i+2)
    - Additionally, store the current size of each subtree in another array with the same indices
    - To insert a value, just find its index, increment the size of the subtree at that index and
      propagate
    - To get the rank of an element, you add up a bunch of subtree counts
    """

    def __init__(self, values):
        """
        Parameters
        ----------
        values: list
            List of sorted (ascending), unique values that will be inserted.
        """
        self._tree = self._treeify(values)
        self._counts = np.zeros_like(self._tree, dtype=int)

    @staticmethod
    def _treeify(values):
        """Convert the np.ndarray `values` into a complete balanced tree.

        Assumes `values` is sorted ascending. Returns a list `t` of the same length in which t[i] >
        t[2i+1] and t[i] < t[2i+2] for all i."""
        if len(values) == 1:  # this case causes problems later
            return values
        tree = np.empty_like(values)
        # Tree indices work as follows:
        # 0 is the root
        # 2n+1 is the left child of n
        # 2n+2 is the right child of n
        # So we now rearrange `values` into that format...

        # The first step is to remove the bottom row of leaves, which might not be exactly full
        last_full_row = int(np.log2(len(values) + 1) - 1)
        len_ragged_row = len(values) - (2 ** (last_full_row + 1) - 1)
        if len_ragged_row > 0:
            bottom_row_ix = np.s_[: 2 * len_ragged_row : 2]
            tree[-len_ragged_row:] = values[bottom_row_ix]
            values = np.delete(values, bottom_row_ix)

        # Now `values` is length 2**n - 1, so can be packed efficiently into a tree
        # Last row of nodes is indices 0, 2, ..., 2**n - 2
        # Second-last row is indices 1, 5, ..., 2**n - 3
        # nth-last row is indices (2**n - 1)::(2**(n+1))
        values_start = 0
        values_space = 2
        values_len = 2 ** last_full_row
        while values_start < len(values):
            tree[values_len - 1 : 2 * values_len - 1] = values[values_start::values_space]
            values_start += int(values_space / 2)
            values_space *= 2
            values_len = int(values_len / 2)
        return tree

    def insert(self, value):
        """Insert an occurrence of `value` into the btree."""
        i = 0
        n = len(self._tree)
        while i < n:
            cur = self._tree[i]
            self._counts[i] += 1
            if value < cur:
                i = 2 * i + 1
            elif value > cur:
                i = 2 * i + 2
            else:
                return
        raise ValueError("Value %s not contained in tree." "Also, the counts are now messed up." % value)

    def __len__(self):
        return self._counts[0]

    def rank(self, value):
        """Returns the rank and count of the value in the btree."""
        i = 0
        n = len(self._tree)
        rank = 0
        count = 0
        while i < n:
            cur = self._tree[i]
            if value < cur:
                i = 2 * i + 1
                continue
            elif value > cur:
                rank += self._counts[i]
                # subtract off the right tree if exists
                nexti = 2 * i + 2
                if nexti < n:
                    rank -= self._counts[nexti]
                    i = nexti
                    continue
                else:
                    return (rank, count)
            else:  # value == cur
                count = self._counts[i]
                lefti = 2 * i + 1
                if lefti < n:
                    nleft = self._counts[lefti]
                    count -= nleft
                    rank += nleft
                    righti = lefti + 1
                    if righti < n:
                        count -= self._counts[righti]
                return (rank, count)
        return (rank, count)

# -*- coding: utf-8 -*-

import numpy as np
# from lifelines.utils.btree import _BTree


def somers_d(event_times, x, event_observed=None) -> float:
    """
    A measure of rank association between [-1, 1] between a censored variable, event_times,
    and another (uncensored) variable, x. -1 is strong anti-correlation, 1 is strong correlation.


    event_times: iterable
         a length-n iterable of observed survival times.
    x: iterable
        a length-n iterable to compare against
    event_observed: iterable, optional
        a length-n iterable censoring flags, 1 if observed, 0 if not. Default None assumes all observed.


    Examples
    --------
    .. code:: python
        from lifelines.datasets import load_rossi
        from lifelines.utils

        T, E = df['week'], df['arrest']
        x = df['age']
        somers_d(T, x, E)


    """
    return 2 * concordance_index(event_times, x, event_observed) - 1


def concordance_index(event_times, predicted_scores, event_observed=None) -> float:
    """
    Calculates the concordance index (C-index) between a series
    of event times and a predicted score. The first is the real survival times from
    the observational data, and the other is the predicted score from a model of some kind.

    The c-index is the average of how often a model says X is greater than Y when, in the observed
    data, X is indeed greater than Y. The c-index also handles how to handle censored values
    (obviously, if Y is censored, it's hard to know if X is truly greater than Y).


    The concordance index is a value between 0 and 1 where:

    - 0.5 is the expected result from random predictions,
    - 1.0 is perfect concordance and,
    - 0.0 is perfect anti-concordance (multiply predictions with -1 to get 1.0)

    The calculation internally done is

    >>> (pairs_correct + 0.5 * pairs_tied) / admissable_pairs

    where ``pairs_correct`` is the number of pairs s.t. if ``t_x > t_y``, then ``s_x > s_y``, pairs,
    ``pairs_tied`` is the number of pairs where ``s_x = s_y``, and ``admissable_pairs`` is all possible pairs. The subtleties
    are in how censored observation are handled (ex: not all pairs can be evaluated due to censoring).


    Parameters
    ----------
    event_times: iterable
         a length-n iterable of observed survival times.
    predicted_scores: iterable
        a length-n iterable of predicted scores - these could be survival times, or hazards, etc. See https://stats.stackexchange.com/questions/352183/use-median-survival-time-to-calculate-cph-c-statistic/352435#352435
    event_observed: iterable, optional
        a length-n iterable censoring flags, 1 if observed, 0 if not. Default None assumes all observed.

    Returns
    -------
    c-index: float
      a value between 0 and 1.

    References
    -----------
    Harrell FE, Lee KL, Mark DB. Multivariable prognostic models: issues in
    developing models, evaluating assumptions and adequacy, and measuring and
    reducing errors. Statistics in Medicine 1996;15(4):361-87.

    Examples
    --------
    .. code:: python

        from lifelines.utils import concordance_index
        cph = CoxPHFitter().fit(df, 'T', 'E')
        concordance_index(df['T'], -cph.predict_partial_hazard(df), df['E'])

    """
    event_times, predicted_scores, event_observed = _preprocess_scoring_data(event_times, predicted_scores, event_observed)
    num_correct, num_tied, num_pairs = _concordance_summary_statistics(event_times, predicted_scores, event_observed)

    return _concordance_ratio(num_correct, num_tied, num_pairs)


def _concordance_ratio(num_correct: int, num_tied: int, num_pairs: int) -> float:
    if num_pairs == 0:
        raise ZeroDivisionError("No admissable pairs in the dataset.")
    return (num_correct + num_tied / 2) / num_pairs


def _concordance_summary_statistics(event_times, predicted_event_times, event_observed):  # pylint: disable=too-many-locals
    """Find the concordance index in n * log(n) time.

    Assumes the data has been verified by lifelines.utils.concordance_index first.
    """
    # Here's how this works.
    #
    # It would be pretty easy to do if we had no censored data and no ties. There, the basic idea
    # would be to iterate over the cases in order of their true event time (from least to greatest),
    # while keeping track of a pool of *predicted* event times for all cases previously seen (= all
    # cases that we know should be ranked lower than the case we're looking at currently).
    #
    # If the pool has O(log n) insert and O(log n) RANK (i.e., "how many things in the pool have
    # value less than x"), then the following algorithm is n log n:
    #
    # Sort the times and predictions by time, increasing
    # n_pairs, n_correct := 0
    # pool := {}
    # for each prediction p:
    #     n_pairs += len(pool)
    #     n_correct += rank(pool, p)
    #     add p to pool
    #
    # There are three complications: tied ground truth values, tied predictions, and censored
    # observations.
    #
    # - To handle tied true event times, we modify the inner loop to work in *batches* of observations
    # p_1, ..., p_n whose true event times are tied, and then add them all to the pool
    # simultaneously at the end.
    #
    # - To handle tied predictions, which should each count for 0.5, we switch to
    #     n_correct += min_rank(pool, p)
    #     n_tied += count(pool, p)
    #
    # - To handle censored observations, we handle each batch of tied, censored observations just
    # after the batch of observations that died at the same time (since those censored observations
    # are comparable all the observations that died at the same time or previously). However, we do
    # NOT add them to the pool at the end, because they are NOT comparable with any observations
    # that leave the study afterward--whether or not those observations get censored.
    if np.logical_not(event_observed).all():
        return (0, 0, 0)

    died_mask = event_observed.astype(bool)
    # TODO: is event_times already sorted? That would be nice...
    died_truth = event_times[died_mask]
    ix = np.argsort(died_truth)
    died_truth = died_truth[ix]
    died_pred = predicted_event_times[died_mask][ix]

    censored_truth = event_times[~died_mask]
    ix = np.argsort(censored_truth)
    censored_truth = censored_truth[ix]
    censored_pred = predicted_event_times[~died_mask][ix]

    censored_ix = 0
    died_ix = 0
    times_to_compare = _BTree(np.unique(died_pred))
    num_pairs = np.int64(0)
    num_correct = np.int64(0)
    num_tied = np.int64(0)

    # we iterate through cases sorted by exit time:
    # - First, all cases that died at time t0. We add these to the sortedlist of died times.
    # - Then, all cases that were censored at time t0. We DON'T add these since they are NOT
    #   comparable to subsequent elements.
    while True:
        has_more_censored = censored_ix < len(censored_truth)
        has_more_died = died_ix < len(died_truth)
        # Should we look at some censored indices next, or died indices?
        if has_more_censored and (not has_more_died or died_truth[died_ix] > censored_truth[censored_ix]):
            pairs, correct, tied, next_ix = _handle_pairs(censored_truth, censored_pred, censored_ix, times_to_compare)
            censored_ix = next_ix
        elif has_more_died and (not has_more_censored or died_truth[died_ix] <= censored_truth[censored_ix]):
            pairs, correct, tied, next_ix = _handle_pairs(died_truth, died_pred, died_ix, times_to_compare)
            for pred in died_pred[died_ix:next_ix]:
                times_to_compare.insert(pred)
            died_ix = next_ix
        else:
            assert not (has_more_died or has_more_censored)
            break

        num_pairs += pairs
        num_correct += correct
        num_tied += tied

    return (num_correct, num_tied, num_pairs)


def _handle_pairs(truth, pred, first_ix, times_to_compare):
    """
    Handle all pairs that exited at the same time as truth[first_ix].

    Returns
    -------
      (pairs, correct, tied, next_ix)
      new_pairs: The number of new comparisons performed
      new_correct: The number of comparisons correctly predicted
      next_ix: The next index that needs to be handled
    """
    next_ix = first_ix
    while next_ix < len(truth) and truth[next_ix] == truth[first_ix]:
        next_ix += 1
    pairs = len(times_to_compare) * (next_ix - first_ix)
    correct = np.int64(0)
    tied = np.int64(0)
    for i in range(first_ix, next_ix):
        rank, count = times_to_compare.rank(pred[i])
        correct += rank
        tied += count

    return (pairs, correct, tied, next_ix)


def _naive_concordance_summary_statistics(event_times, predicted_event_times, event_observed):
    """
    Fallback, simpler method to compute concordance.

    """

    def _valid_comparison(time_a, time_b, event_a, event_b):
        """True if times can be compared."""
        if time_a == time_b:
            # Ties are only informative if exactly one event happened
            return event_a != event_b
        if event_a and event_b:
            return True
        if event_a and time_a < time_b:
            return True
        if event_b and time_b < time_a:
            return True
        return False

    def _concordance_value(time_a, time_b, pred_a, pred_b, event_a, event_b):
        if pred_a == pred_b:
            # Same as random
            return (0, 1)
        if pred_a < pred_b:
            return (time_a < time_b) or (time_a == time_b and event_a and not event_b), 0
        # pred_a > pred_b
        return (time_a > time_b) or (time_a == time_b and not event_a and event_b), 0

    num_pairs = 0.0
    num_correct = 0.0
    num_tied = 0.0

    for a, time_a in enumerate(event_times):
        pred_a = predicted_event_times[a]
        event_a = event_observed[a]
        # Don't want to double count
        for b in range(a + 1, len(event_times)):
            time_b = event_times[b]
            pred_b = predicted_event_times[b]
            event_b = event_observed[b]

            if _valid_comparison(time_a, time_b, event_a, event_b):
                num_pairs += 1.0
                crct, ties = _concordance_value(time_a, time_b, pred_a, pred_b, event_a, event_b)
                num_correct += crct
                num_tied += ties

    return (num_correct, num_tied, num_pairs)


def naive_concordance_index(event_times, predicted_event_times, event_observed=None) -> float:
    event_times, predicted_event_times, event_observed = _preprocess_scoring_data(
        event_times, predicted_event_times, event_observed
    )
    return _concordance_ratio(*_naive_concordance_summary_statistics(event_times, predicted_event_times, event_observed))


def _preprocess_scoring_data(event_times, predicted_scores, event_observed):
    event_times = np.asarray(event_times, dtype=float)
    predicted_scores = np.asarray(predicted_scores, dtype=float)

    # Allow for (n, 1) or (1, n) arrays
    if event_times.ndim == 2 and (event_times.shape[0] == 1 or event_times.shape[1] == 1):
        # Flatten array
        event_times = event_times.ravel()
    # Allow for (n, 1) or (1, n) arrays
    if predicted_scores.ndim == 2 and (predicted_scores.shape[0] == 1 or predicted_scores.shape[1] == 1):
        # Flatten array
        predicted_scores = predicted_scores.ravel()

    if event_times.shape != predicted_scores.shape:
        raise ValueError("Event times and predictions must have the same shape")
    if event_times.ndim != 1:
        raise ValueError("Event times can only be 1-dimensional: (n,)")

    if event_observed is None:
        event_observed = np.ones(event_times.shape[0], dtype=float)
    else:
        event_observed = np.asarray(event_observed, dtype=float).ravel()
        if event_observed.shape != event_times.shape:
            raise ValueError("Observed events must be 1-dimensional of same length as event times")

    # check for NaNs
    for a in [event_times, predicted_scores, event_observed]:
        if np.isnan(a).any():
            raise ValueError("NaNs detected in inputs, please correct or drop.")

    return event_times, predicted_scores, event_observed

In [2]:
### --- iPython Config --- ###
from IPython import get_ipython
if 'IPython.extensions.autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')
else:
    get_ipython().run_line_magic('reload_ext', 'autoreload')
%autoreload 2
### --- System and Path --- ###
import os
import sys
repo_path = os.path.dirname(os.getcwd())
if repo_path not in sys.path:
    sys.path.append(repo_path)
import warnings
warnings.filterwarnings('ignore')

# %%
import pandas as pd
import numpy as np
# from lifelines.utils import concordance_index

# Load data
# local:
df_train = pd.read_csv(os.path.join(repo_path, 'data', 'raw', 'train.csv'))
df_test = pd.read_csv(os.path.join(repo_path, 'data', 'raw', 'test.csv'))
# kaggle:
# df_train = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
# df_test = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

# %%
# drop columns
cols = ['ID']
id_df_test = df_test['ID']  # For submission
df_train.drop(cols, axis=1, inplace=True)
df_test.drop(cols, axis=1, inplace=True)

# %%
target = 'efs'
time_col = 'efs_time'  # Survival time column

X_train = df_train.drop([target, time_col], axis=1)
y_train = df_train[target]
efs_time = df_train[time_col]  # Extract survival times for C-index calculation

X_test = df_test  # No target

# %%
# OneHotEncoding
from sklearn.preprocessing import OneHotEncoder

# Identify categorical and numerical columns
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Apply OneHotEncoding
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = ohe.fit_transform(X_train[cat_cols])
X_test_cat = ohe.transform(X_test[cat_cols])

# Convert OHE to DataFrame and maintain column names
X_train_cat = pd.DataFrame(X_train_cat, columns=ohe.get_feature_names_out(), index=X_train.index)
X_test_cat = pd.DataFrame(X_test_cat, columns=ohe.get_feature_names_out(), index=X_test.index)

# Concatenate with Numerical columns
X_train_final = pd.concat([X_train[num_cols].reset_index(drop=True), X_train_cat.reset_index(drop=True)], axis=1)
X_test_final = pd.concat([X_test[num_cols].reset_index(drop=True), X_test_cat.reset_index(drop=True)], axis=1)

# Ensure the same feature set between train and test
X_test_final = X_test_final.reindex(columns=X_train_final.columns, fill_value=0)

# %%
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

def objective(trial):
    """
    Optuna optimization using C-index as the evaluation criterion.
    """
    # --- 3.1. Suggest hyperparameters ---
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'random_strength': trial.suggest_float('random_strength', 1e-2, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 256),
        'verbose': 0,
        'random_state': 42
    }

    # --- 3.2. Define the model ---
    model = CatBoostClassifier(**param)

    # --- 3.3. Evaluate using cross-validation ---
    n_splits = 5
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    c_index_scores = []

    for train_idx, val_idx in cv.split(X_train_final, y_train):
        X_tr, X_val = X_train_final.iloc[train_idx], X_train_final.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        efs_time_val = efs_time.iloc[val_idx]  # Get survival times for validation

        # Train model
        model.fit(X_tr, y_tr, verbose=0)

        # Predict risk scores (probabilities)
        y_pred_proba = model.predict_proba(X_val)[:, 1]  # Higher values mean higher risk

        # Compute Concordance Index
        c_index = concordance_index(efs_time_val, -y_pred_proba, y_val)  # Negative for risk interpretation
        c_index_scores.append(c_index)

    return np.mean(c_index_scores)  # Maximizing mean C-index

# Create Optuna study
study = optuna.create_study(direction='maximize')  # We want to maximize C-index
n_trials = 7
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

# Print results
print("Number of finished trials:", len(study.trials))
print("Best trial:")
trial_ = study.best_trial

print("  Best C-index:", trial_.value)
print("  Params: ")
for key, value in trial_.params.items():
    print(f"    {key}: {value}")

# Train the final model with best parameters
best_params = study.best_params
final_model = CatBoostClassifier(
    **best_params,
    verbose=100,
    random_state=42
)

final_model.fit(X_train_final, y_train)

# %%
# Save the model (optional)
# import joblib
# joblib.dump(final_model, '../models/catboost_model.pkl')

# %%
# Inference
y_pred_proba = final_model.predict_proba(X_test_final)[:, 1]  # Probability of event occurring

# %%
# Save submission
submission = pd.DataFrame({'ID': id_df_test, 'prediction': y_pred_proba})
# submission.to_csv(os.path.join(repo_path, 'data', 'submission', 'submission.csv'), index=False)
submission.to_csv('submission.csv', index=False)
print("Submission file saved successfully!")
submission

[I 2025-02-02 17:18:34,379] A new study created in memory with name: no-name-ed393adb-09a0-4d24-96eb-a3c38aa2bd01


  0%|          | 0/7 [00:00<?, ?it/s]

[I 2025-02-02 17:18:49,127] Trial 0 finished with value: 0.6542372019813001 and parameters: {'iterations': 115, 'depth': 9, 'learning_rate': 0.1408039263778209, 'l2_leaf_reg': 2.2347991414391246, 'random_strength': 0.14944250659759895, 'bagging_temperature': 0.5511433935825152, 'border_count': 43}. Best is trial 0 with value: 0.6542372019813001.
[I 2025-02-02 17:18:55,548] Trial 1 finished with value: 0.64887670763244 and parameters: {'iterations': 115, 'depth': 6, 'learning_rate': 0.022210486851574358, 'l2_leaf_reg': 3.147913909689499, 'random_strength': 2.050425329029774, 'bagging_temperature': 0.26555598707441086, 'border_count': 234}. Best is trial 0 with value: 0.6542372019813001.
[I 2025-02-02 17:19:28,383] Trial 2 finished with value: 0.6622717438107232 and parameters: {'iterations': 719, 'depth': 6, 'learning_rate': 0.01884892696979274, 'l2_leaf_reg': 7.129540530307217, 'random_strength': 1.0704062122598603, 'bagging_temperature': 0.1596179759584443, 'border_count': 91}. Best i

,ID,prediction
0,28800,0.169306
1,28801,0.644905
2,28802,0.077094
